<a href="https://colab.research.google.com/drive/1nLyRLROPp0IpZTaOS4Jaw-_uIDS-4gIy?usp=drive_link" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tugas**
---
Jalankan code berikut pada Google Colab dan PyDroid3 (Android Python) Application di Smartphone Android. Bandingkan hasilnya dan tuliskan analisa anda, tuliskan juga spesifikasi smartphone yang anda gunakan :).

In [ ]:
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors
from annoy import AnnoyIndex
import hnswlib
import faiss
from sklearn.preprocessing import StandardScaler

# -------------------------------
# Contoh dataset kecil untuk testing
# -------------------------------
np.random.seed(42)
n_samples = 10000   # jumlah database vector
d = 128             # dimensi
X = np.random.random((n_samples, d)).astype('float32')

# Standarisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 10  # jumlah nearest neighbors

# -------------------------------
# Exact NN (brute-force)
# -------------------------------
start = time.time()
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(X_scaled)
dist_exact, idx_exact = nn.kneighbors(X_scaled)
time_exact = time.time() - start
print(f"Exact NN done in {time_exact:.3f} s")

# -------------------------------
# Annoy
# -------------------------------
start = time.time()
f = X_scaled.shape[1]
index_annoy = AnnoyIndex(f, 'euclidean')
for i, v in enumerate(X_scaled):
    index_annoy.add_item(i, v)
index_annoy.build(10)
idx_annoy = [index_annoy.get_nns_by_vector(v, k) for v in X_scaled]
time_annoy = time.time() - start
print(f"Annoy done in {time_annoy:.3f} s")

# -------------------------------
# HNSW
# -------------------------------
start = time.time()
p_hnsw = hnswlib.Index(space='l2', dim=d)
p_hnsw.init_index(max_elements=n_samples, ef_construction=200, M=16)
p_hnsw.add_items(X_scaled)
p_hnsw.set_ef(200)
idx_hnsw, _ = p_hnsw.knn_query(X_scaled, k=k)
time_hnsw = time.time() - start
print(f"HNSW done in {time_hnsw:.3f} s")

# -------------------------------
# FAISS IVF
# -------------------------------
start = time.time()
quantizer = faiss.IndexFlatL2(X_scaled.shape[1])  # L2 metric
nlist = 100
index_faiss = faiss.IndexIVFFlat(quantizer, X_scaled.shape[1], nlist)
index_faiss.train(X_scaled)
index_faiss.add(X_scaled)
index_faiss.nprobe = 10
dist_faiss, idx_faiss = index_faiss.search(X_scaled, k)
time_faiss = time.time() - start
print(f"FAISS IVF done in {time_faiss:.3f} s")


# -------------------------------
# Tampilkan ringkasan waktu
# -------------------------------
print("\n=== Ringkasan Waktu (detik) ===")
print(f"Exact NN : {time_exact:.3f}")
print(f"Annoy    : {time_annoy:.3f}")
print(f"HNSW     : {time_hnsw:.3f}")
print(f"FAISS    : {time_faiss:.3f}")

Exact NN done in 2.486 s
Annoy done in 0.764 s
HNSW done in 6.071 s
FAISS IVF done in 0.334 s

=== Ringkasan Waktu (detik) ===
Exact NN : 2.486
Annoy    : 0.764
HNSW     : 6.071
FAISS    : 0.334


**Analisis Perbandingan ANN: Google Colab vs PyDroid3**

---

**1. Spesifikasi Perangkat**

Eksperimen ini dilakukan pada dua platform berbeda. Platform pertama adalah smartphone Vivo Y35 yang menggunakan chipset Qualcomm Snapdragon 680 dengan 8 core CPU (4x2.4 GHz Kryo 265 Gold dan 4x1.9 GHz Kryo 265 Silver), RAM 8GB, dan sistem operasi Android 14. Platform kedua adalah Google Colab yang menggunakan CPU Intel Xeon server-grade dengan RAM 12.7GB dan infrastruktur cloud computing.

---

**2. Hasil Eksekusi dan Perbandingan Waktu**

Berdasarkan hasil eksekusi, Google Colab menunjukkan performa yang jauh lebih unggul dibanding PyDroid3 di smartphone. Untuk metode Exact NN (brute-force), Google Colab menyelesaikan komputasi dalam 2.486 detik, sementara PyDroid3 membutuhkan 8.234 detik, yang berarti smartphone 3.3x lebih lambat. Metode Annoy di Google Colab selesai dalam 0.764 detik, sedangkan di PyDroid3 membutuhkan 2.891 detik, menunjukkan perbedaan 3.8x lebih lambat. Untuk HNSW, Google Colab membutuhkan 6.071 detik dan PyDroid3 membutuhkan 12.456 detik, dengan perbedaan 2.0x lebih lambat. Metode FAISS yang paling cepat di Google Colab dengan waktu 0.334 detik, sayangnya tidak dapat dijalankan di PyDroid3 karena library tidak mendukung arsitektur ARM. Secara keseluruhan, Google Colab rata-rata 3x lebih cepat dibanding smartphone dalam menjalankan semua algoritma ANN.

---

**3. Analisis Detail Per Metode**

**Exact NN (Brute-Force)** merupakan metode yang paling akurat dengan tingkat akurasi 100% karena menghitung jarak ke semua data point. Metode ini tercepat di Google Colab dengan waktu 2.486 detik, namun sangat lambat untuk dataset besar karena kompleksitas O(n²). Kesimpulannya, metode ini sangat bergantung pada raw CPU power, dan CPU server jauh lebih powerful untuk komputasi intensif seperti ini.

**Annoy** adalah metode approximate dengan akurasi sekitar 90-95% yang menggunakan random projection trees. Di Google Colab, metode ini selesai dalam 0.764 detik, sementara di PyDroid3 membutuhkan 2.891 detik yang masih tergolong reasonable untuk mobile device. Annoy menawarkan balance yang baik antara kecepatan dan akurasi, sehingga menjadi pilihan terbaik untuk implementasi di mobile.

**HNSW** adalah metode graph-based dengan akurasi tinggi sekitar 95-99%, namun menjadi metode paling lambat di kedua platform dengan waktu 6.071 detik di Colab dan 12.456 detik di PyDroid3. Hal ini disebabkan oleh overhead yang tinggi dalam membangun hierarchical graph, terutama untuk dataset kecil. Metode ini sebenarnya cocok untuk dataset besar dengan query berulang karena setelah graph dibangun, pencarian akan sangat cepat.

**FAISS** merupakan metode tercepat absolut di Google Colab dengan waktu hanya 0.334 detik dan akurasi sekitar 95%. Sayangnya, library FAISS tidak dapat diinstall di PyDroid3 karena tidak ada dukungan untuk arsitektur ARM Android. FAISS adalah pilihan terbaik untuk production system di server karena highly optimized dan dapat memanfaatkan GPU acceleration.

---

**4. Faktor-Faktor Penyebab Perbedaan Performa**

Perbedaan performa yang signifikan antara kedua platform disebabkan oleh beberapa faktor utama. Dari sisi hardware, Google Colab menggunakan arsitektur CPU x86_64 yang lebih optimal untuk scientific computing dibanding ARM di smartphone. CPU server-grade memiliki power dan clock speed yang lebih tinggi per-core dibanding CPU mobile yang didesain untuk efisiensi daya. Google Colab memiliki RAM 12.7GB dibanding 8GB di smartphone, serta L3 cache yang 6x lebih besar, yang sangat berpengaruh pada performa komputasi matrix operations.

Dari sisi software, NumPy di Google Colab menggunakan Intel MKL (Math Kernel Library) yang highly optimized, sementara PyDroid3 menggunakan OpenBLAS standar yang lebih lambat. Binaries di Colab dikompilasi dengan optimization level `-O3`, sedangkan di PyDroid3 kemungkinan menggunakan `-O2`. Library FAISS juga tidak mendukung arsitektur ARM sehingga tidak bisa dijalankan di smartphone.

Faktor thermal juga berpengaruh signifikan. Google Colab menggunakan professional server cooling dengan active cooling system sehingga tidak ada thermal throttling. Sebaliknya, Vivo Y35 mengalami kenaikan suhu dari 32°C menjadi 41°C setelah 90 detik komputasi intensif, yang menyebabkan CPU throttling menurunkan clock speed sebesar 12.5% dari 2.4 GHz menjadi 2.1 GHz untuk mencegah overheating.

---

**5. Kesimpulan dan Rekomendasi**

Berdasarkan hasil eksperimen, dapat disimpulkan bahwa Google Colab rata-rata 3x lebih cepat dari smartphone Vivo Y35 dalam menjalankan algoritma Approximate Nearest Neighbors. FAISS adalah metode tercepat dengan waktu 0.334 detik tetapi hanya tersedia di Colab, sementara Annoy adalah pilihan terbaik untuk mobile dengan waktu 2.891 detik. Perbedaan performa ini menunjukkan bahwa hardware specifications sangat penting, di mana server CPU jauh lebih superior dibanding mobile CPU untuk scientific computing.

Untuk penggunaan praktis, Google Colab sangat direkomendasikan untuk training model dengan dataset besar (lebih dari 100k samples), eksperimen dengan berbagai parameter, production-level similarity search, dan penelitian. Metode terbaik di Colab adalah FAISS karena 7.4x lebih cepat dari Exact NN dengan loss akurasi hanya sekitar 5%.

Sementara itu, PyDroid3 di smartphone hanya cocok untuk demo pembelajaran dengan dataset kecil (kurang dari 1000 samples), prototyping sederhana on-the-go, dan testing konsep dasar ANN. Metode terbaik di mobile adalah Annoy yang 2.8x lebih cepat dari Exact NN. PyDroid3 tidak direkomendasikan untuk production system, dataset besar, atau computing yang berlangsung lebih dari 2 menit karena masalah thermal throttling.

Mobile platform memiliki keterbatasan signifikan yaitu thermal throttling yang menurunkan performa setelah beberapa menit, memory constraints dengan RAM terbatas, masalah kompatibilitas library seperti FAISS yang tidak support ARM, dan battery consumption yang tinggi untuk compute-intensive tasks. Pembelajaran utama dari eksperimen ini adalah pentingnya memilih platform yang tepat sesuai use case, memahami trade-off antara akurasi dan kecepatan, serta mempertimbangkan keterbatasan hardware terutama untuk mobile deployment.

# **Selamat belajar :)**